In [1]:
import numpy as np
import math
import random
import os
import pybullet as p
import pybullet_data
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import matplotlib.pyplot as plt
import time

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Concatenate
from keras.optimizers import Adam

C:\Users\SERVER\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framew

In [3]:
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import  SequentialMemory

In [4]:
class BikeEnv(gym.Env):
    def __init__(self):
        p.connect(p.GUI)
        p.resetDebugVisualizerCamera(cameraDistance=10, cameraYaw=0, cameraPitch=-40, cameraTargetPosition=[0.55,-0.35,0.2])
        self.action_space = spaces.Discrete(5)
        self.observation_space = spaces.Box(np.array([-1000]*10), np.array([1000]*10))
        self.timestep = 1./240.
        
    def step(self, action):
        #print(action,)
        print(p.getJointState(self.pid,1)[1])
        
        if action==0 :
            self.v = 0
        elif action==1 :
            self.v = self.v - 5
            if self.v < 0 : self.v =0 
        elif action==2 : 
            self.v = self.v - 10
            if self.v < 0 : self.v =0 
        elif action==3 : 
            self.v = self.v + 5
            if self.v > 100 : self.v = 100 
        else :
            self.v = self.v + 10
            if self.v > 100 : self.v = 100
                
        p.setJointMotorControl2(self.pid, 2, p.VELOCITY_CONTROL, targetVelocity=self.v, force=20)
        
       # if action[1] < 0.5 :   
       #     p.setJointMotorControl2(self.pid, 2, p.VELOCITY_CONTROL, targetVelocity=0, force=0) 
       # else:  
       #     p.setJointMotorControl2(self.pid, 2, p.VELOCITY_CONTROL, targetVelocity=5, force=5)    
    
        p.stepSimulation()
        #time.sleep(self.timestep)
        
        state = p.getLinkState(self.pid,0)[0]
        if state[2] <= 0.7 or  state[2] >= 2:
            reward = -1000
            done = True
        else :
            #reward = math.sqrt((self.origin[0]-state[0])**2+(self.origin[1]-state[1])**2)
            reward = 1
            done = False
        self.origin = state 
        
        velocity = p.getBaseVelocity(self.pid)
        state_object ,orient = p.getBasePositionAndOrientation(self.pid)
        norm = math.sqrt(orient[0]**2+orient[1]**2+orient[2]**2+orient[3]**2)
        observation =velocity[0]+velocity[1]+ orient
        
        info = {'x':state_object[0],'y':state_object[1],'z':state_object[2]}
        return observation, reward, done, info
            
        
    def reset(self):
        p.resetSimulation()
        self.v = 10 ;
        urdfRootPath = pybullet_data.getDataPath()
        planeUid = p.loadURDF(os.path.join(urdfRootPath,"plane.urdf"), basePosition=[0,0,0])
        self.pid = p.loadURDF(os.path.join(urdfRootPath, "bicycle/bike.urdf"),basePosition=[0,0,1])
        self.origin = p.getLinkState(self.pid,0)[0]
        p.setGravity(0,0,-10)
        #p.setRealTimeSimulation(0)
        p.setJointMotorControl2(self.pid, 1, p.VELOCITY_CONTROL, force=0) 
        
        velocity = p.getBaseVelocity(self.pid)
        state_object ,orient = p.getBasePositionAndOrientation(self.pid)
        norm = math.sqrt(orient[0]**2+orient[1]**2+orient[2]**2+orient[3]**2)
        observation =velocity[0]+velocity[1]+ orient
        
        p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
        return observation
        
    def render(self, mode='human'):
        view_matrix = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0.7,0,0.05],
                                                            distance=.7,
                                                            yaw=90,
                                                            pitch=-70,
                                                            roll=0,
                                                            upAxisIndex=2)
        proj_matrix = p.computeProjectionMatrixFOV(fov=60,
                                                     aspect=float(960) /720,
                                                     nearVal=0.1,
                                                     farVal=100.0)
        (_, _, px, _, _) = p.getCameraImage(width=960,
                                              height=720,
                                              viewMatrix=view_matrix,
                                              projectionMatrix=proj_matrix,
                                              renderer=p.ER_BULLET_HARDWARE_OPENGL)

        rgb_array = np.array(px, dtype=np.uint8)
        rgb_array = np.reshape(rgb_array, (720,960, 4))

        rgb_array = rgb_array[:, :, :3]
        return rgb_array

    def close(self):
        p.disconnect()

In [5]:
env = BikeEnv()
np.random.seed(123)
env.seed(123)

C:\Users\SERVER\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
nb_actions = env.action_space.n

In [7]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                176       
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_3 (Activation)    (None, 16)               

In [8]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [9]:
dqn.fit(env, nb_steps=50000, visualize=True, verbose=2)

Training for 50000 steps ...

0.0
1.659110602170943e-24
1.7433884733252233e-18
8.716795461275455e-19
0.023846773038626676
8.079566059839005e-07
8.077824666142042e-07
0.02384758064863733
1.1542772098034615e-06
0.38958585829194303
0.4423923655587251
0.47397436898131534


C:\Users\SERVER\Anaconda3\lib\site-packages\rl\memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


0.5318304241021984
0.08533280141107064
-0.04117294946357475
-0.016103496581955822
-0.000581717795938309
-0.00027694873646659366
0.024775232120152185
0.010356273709783592
0.03427684611303475
0.02046843012162609
0.0432417868542493
0.06827984587754114
0.09331827248238601
0.11834700250655153
0.14336055108151338
0.16835080195133367
0.19332474211968154
0.21828158698056352
0.24322119834999603
0.26814112750205643
0.269052857015182
0.25317196470213
0.25098737674128385
0.24993047305073984
0.23420875573200328
0.21866913460303408
0.22047213184629552
0.24547760370549174
0.24412204335806667
0.24355114973001124
0.24185565499340206
0.24171306808242995
0.26671860189567
0.2916311351830445
0.31654133475223384
0.34143783959035034
0.36633497213949723
0.3912008552865964
0.41607342706806716
0.44094695891661173
0.46582819675378506
0.46403201642084163
0.4607037996275961
0.4463691759244695
0.4317631294119846
0.4168409888881728
0.44181245073387115
0.46668405462513074
0.4915756305423267
0.5164964249802071
0.54145

0.46255301994412956
0.4885655785489202
0.47021472167947825
0.4509332306274623
0.432989546133283
0.41662417995007384
0.42061113012985607
0.44697324486606227
0.4734918496986219
0.5002885944728502
0.48492352173487024
0.46483877205891644
0.44462374284851364
0.42815583757020875
0.41261484716107616
0.3971914470247139
0.4054928775981221
0.4325548887346954
0.4598183846373975
0.4403042870764101
0.4201848225189232
0.442054049094852
0.4696552375028537
0.4975152219017206
0.5258111081507475
0.5546568426256197
0.5841985688458254
0.5823788446361744
0.5595433470020607
0.5140306273675779
0.5117261302166355
0.49743708422287897
0.4832231278376621
0.45353266453735597
0.47778082450541004
0.4830510089946624
0.4799582283264189
0.45291289201402596
0.4576545715842124
0.44970098481572385
0.4539087481917651
0.4468341561545089
0.4505351110149158
0.4452754296308483
0.43233304923506505
0.4125905806780283
0.4367358539512368
0.46085852534696115
0.48497956953016275
0.4871980287080131
0.48656180948644606
0.464284311913

0.01342009680139603
0.03722381686679791
0.026948282060841235
0.050315920019988664
0.07436884012936935
0.0984562303546381
0.09268636761930776
0.0802704098912564
0.06794564752911948
0.06115527279807224
0.08287437247657344
0.07356841600118244
0.08912596317549618
0.08006325239353815
0.06808930001224382
0.05598795966261568
0.04391502840834994
0.03189042040680835
0.05471767717555436
0.07851702058400645
0.07124345104729421
0.05924558292045317
0.04729947373022987
0.03536160560135304
0.023442918757254384
0.01155427717547587
0.00037833964781203155
0.0002733075927733483
0.00021871936792186494
0.00018889131580283804
0.0001770987587337007
0.023737791550931336
0.014263776154150339
0.002549773872992366
0.00029725838310185266
0.0002815580087339201
0.00029005210056351846
0.00029974073561530044
0.0003132552965772837
0.0002975253427823975
0.0003167092014804662
0.0002969491859704137
0.023520713242575087
0.0467849210760689
0.040444553837975336
0.028940415922623225
0.01771605985151002
0.006633209567625071
0

...

0.9588320763472022
0.982527532315752
1.0069327216186639
0.9995827931629367
0.9890397646988749
0.9773289539888291
0.9649988947620187
0.9515906630663256
0.9311239456545032
0.9391312791003578
0.9296848350911363
0.9175025989518908
0.9140721939772383
0.9039405767041008
0.9181805427245355
0.9070554566916929
0.9229255748125731
0.9490585858585923
1.0559539783545033
1.0813881444386526
1.10766655520873
1.1348270709344257
1.163118891922088
1.192752203970713
1.143919534955232
1.1660755526507423
1.1884189446030702
1.2108908571386694
1.2343529112272642
1.2574539043610058
1.2805887953382504
1.3026868807555918
1.3093122932785062
1.3311286368029727
1.3540528921891648
1.3482888971345017
1.3381195668753096
1.3330643774429431
1.3571272993686883
1.3812837569101146
1.4061288698874816
1.4318436939058445
1.4316452417418648
1.4167676316373077
1.40096028864486
1.4069104063294278
1.5472538938661757
1.5715754014976584
1.5495895939964661
1.5747218622555994
1.4140405409306096
1.4196630179756324
1.4076838988934424
1

0.812124684079962
0.8311109791579386
0.8565277743636903
0.8457256742703237
0.8333860594703731
0.8404355775497431
0.829487689117297
0.9132063346111857
0.8938894447085697
0.8726191153083814
0.8510505648568453
0.8306780131742705
0.8151999874735117
0.8123252477040265
0.8256218182106773
0.8521496071603628
0.8796864768498107
0.860515714037977
0.8393939376725535
0.8180651899460465
0.8281503310779532
0.7874686762625699
0.7966503531275562
0.8026322177816299
0.7955428232899483
0.7832353093324308
0.7710545919250368
0.7589221868964106
0.7468635406756994
0.6976866532670528
0.7051016934874824
0.673564833124018
0.6811819730710593
0.68643474811931
0.677893095267498
0.6681168651575496
0.6582832881097906
0.6484300428612222
0.6385944136579412
0.6288064913308755
0.619090560650958
0.5811961434118131
0.5883343662091558
0.5876902797397715
0.5511526175226956
0.5652653570062957
0.5849647831539517
0.6059077015879623
0.6275561512003679
0.6357289852041658
0.6478805606287745
0.6514185228425113
0.6536781874080182
0

1.5003682837272108
1.518879581861877
1.5353907064960934
1.54920550691483
1.5482905322045335
1.5608431648900445
1.5619267284671823
1.5528848772501824
1.5638881731906535
1.5797525443498106
1.5784031432040446
1.4878163860751288
1.5411778518048735
1.5610287594581214
1.578048972217063
1.552812953663304
1.5534460831987313
1.5585928007794922
1.5683502731374181
1.5772558969369768
1.5893951929756134
1.6362211949673382
1.637058153318059
1.6328647306284823
1.6281143982571467
1.6227731223063835
1.650185839867659
1.6778109272485762
1.6350330516175506
1.6131692978116854
1.5955893766436604
1.5775277027654158
1.559782890661824
1.5524541966260532
1.6210624413011736
1.603288603157038
1.596650814837345
1.6103245622743447
1.5888750012480872
1.5903212693007107
1.589910473974552
1.582457188772989
1.5982708759078958
1.571671734641595
 49029/50000: episode: 150, duration: 17.099s, episode steps: 328, steps per second: 19, episode reward: -673.000, mean reward: -2.052 [-1000.000, 1.000], mean action: 2.140 [0.

0.2908495292363472
0.2754823828988484
0.27413157685007944
0.2991489741773075
0.32407435731281575
0.3490138127027227
0.3463469905366343
0.33582503736606234
0.3208410974214047
0.31642133238857917
0.34143655299831277
0.3663981122746765
0.3913707374833
0.4163600886125536
0.4413753348742659
0.46642765480417475
0.49153061414445703
0.5167006385194974
0.541957588944798
0.5673254653131854
0.5928332709777906
0.618516081405624
0.6444163727439184
0.6705856831811488
0.6970867027841683
0.7239714835645428
0.7514142815434309
0.7794455325288748
0.8082252583320253
0.8379448993722215
0.8688227055984314
0.8794319087251732
0.9072997479692001
0.9126016221728371
0.9395953280419156
0.9688487778620005
0.9276835386715137
0.9516876992466956
1.0237032237973156
1.0223560484379013
0.9836032767323459
1.0057747769224008
1.0041164391095412
1.0013527804847466
1.0246813835608717
1.0480443836788667
1.0715084742257617
1.0951190649491416
1.1189234896463858
1.1429789395097247
1.1673547491287701
1.1929623143370693
1.21889046

0.867175041753314
0.8626434406316199
0.8556291941781999
0.8428346854289861
0.8675710965091238
0.8921739444733294
0.8857331628927821
0.8767286503777796
0.8762956310081722
0.9014527430656505
0.9266403327927152
0.9521970351812181
1.061895815033111
1.0446939396246062
1.0248880979200756
1.0040982312987707
0.9826650729400006
0.9610537908080848
0.9515573903276139
0.965477281078031
0.9912776674406426
1.0175236499168
1.044555202366402
1.0725092816943715
1.1016068312566627
1.1321112767395611
1.0778005034612523
1.1002341099344528
1.1229941707044884
1.1459355742754511
1.1690741677769225
1.1911700636718845
1.2144220386771898
1.2385827353037573
1.26322402834004
1.277342408283887
1.2722516144896794
1.2646332489676158
1.2551801214623977
1.2337538929183007
1.2256953096560887
1.2157172316749418
1.2022378950041377
1.2267902701461393
1.3074599524799184
1.3281582635884568
1.3122461675921362
1.2910425825365164
1.290929161051395
1.3163758047438003
1.3424309199466167
1.3249361650830969
1.3505064290843285
1.37